In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_excel("../data/Train.xlsx")

In [3]:
df

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [5]:
## Prepare proposal document and final presentation.

In [6]:
# Question:
## 5w and h ## 
# 1. Which airline is most preffered airline.
# 2. Mean/Median price for which airline is highest and lowest.
# 3. For each flight which is the most common source and destination airport.
# 4. How many flights are going from each source airports and destination airport.
# 5. Which airline is having most non-stop flight.
# 6. Which day of week we have maximum and minimum flights?
# 7. During what time day we have more flights?
# 8. What is the average price for flights on each day of week?
#      a. Different hours of day [morning/afternoon/evening/night] what is the average price?
# 9. How price and duration of flight is varring?
# 10. For which source and destiantion you have maximum and minimum duration?

# Sanity Check

In [7]:
# check for duplicate data
# Date of Jorney is object need to be converted into Datetime.
# Dep_Time is fine.
# Arrival Time is incorrectly placed so need to fixed.
# Duration is available in HH:MM format need to be either in minutes or hours. checked
# Total_Stops should be numerical but right it is object.
# Check if Additional_Info column provides any meaning information or not.

In [7]:
## Drop Duplicates
df.drop_duplicates(inplace=True)

In [8]:
def time_minute(duration):
    dur = duration.split()
    min = 0
    for x in dur:
        if x[-1] == 'h':
            min = min + int(x[:-1])*60

        if x[-1] == 'm':
            min = min + int(int(x[:-1])) 
    return min

def convert_duration_to_minutes(time):
    '''
    This function converts duration in h m to minutes:
    input : hh:mm ,hh, mm
    return:
        min
    '''
    if len(time.split(' ')) >1 :
        hh,mm = time.split(' ')
        hh,mm = int(hh[:-1]),int(mm[:-1])
        duration = hh*60+mm
    else:
        if 'h' in time:
            duration = int(time[:-1])*60
        else:
            duration= int(time[:-1])
            
    return duration

In [9]:
# df['DepartureDateTime'] = df['Date_of_Journey'] + " "+ df['Dep_Time']
# df['DepartureDateTime'] = pd.to_datetime(df['DepartureDateTime'],infer_datetime_format=True)
# df['Duration_min'] = df['Duration'].apply(lambda x: convert_duration_to_minutes(x))
# df['Duration_timedelta'] = pd.to_timedelta(df['Duration_min'], unit='m')
# df["ArrivalDateTime"] = df['DepartureDateTime'] + df['Duration_timedelta']

def create_arrival_time(df):
    '''
    This Function preprocess date_of_journey and duration to create departure and arrival date time.
    '''
    df['DepartureDateTime'] = df['Date_of_Journey'] + " "+ df['Dep_Time']
    df['DepartureDateTime'] = pd.to_datetime(df['DepartureDateTime'],infer_datetime_format=True)
    df['Duration_min'] = df['Duration'].apply(lambda x: convert_duration_to_minutes(x))
    df['Duration_timedelta'] = pd.to_timedelta(df['Duration_min'], unit='m')
    df["ArrivalDateTime"] = df['DepartureDateTime'] + df['Duration_timedelta']
    return df

create_arrival_time(df)

In [14]:
df.drop(columns=['Date_of_Journey','Dep_Time','Arrival_Time','Duration'],axis=1,inplace=True)

In [15]:
stops_dict = {
    'non-stop':0,
    '2 stops':2,
    '1 stop':1,
    '3 stops':3,
    '4 stops':4
}
df['Total_Stops'] = df['Total_Stops'].replace(stops_dict)

In [16]:
df['Additional_Info'].value_counts()/len(df)*100

No info                         78.208927
In-flight meal not included     18.407722
No check-in baggage included     3.039281
1 Long layover                   0.181592
Change airports                  0.066902
Business class                   0.038230
No Info                          0.028672
1 Short layover                  0.009557
Red-eye flight                   0.009557
2 Long layover                   0.009557
Name: Additional_Info, dtype: float64

In [17]:
# Additional Info columns doesnot have any meaningful information.
df.drop(columns=['Additional_Info'],axis=1,inplace=True)

In [22]:
df.head(2)

,Airline,Source,Destination,Route,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime
0,IndiGo,Banglore,New Delhi,BLR → DEL,0.0,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2.0,7662,2019-05-01 05:50:00,445,0 days 07:25:00,2019-05-01 13:15:00


#### Actions on Sanity Check

In [ ]:
def convert_duration_to_minutes(time):
    '''
    This function converts duration in h m to minutes:
    input : hh:mm ,hh, mm
    return:
        min
    '''
    if len(time.split(' ')) >1 :
        hh,mm = time.split(' ')
        hh,mm = int(hh[:-1]),int(mm[:-1])
        duration = hh*60+mm
    else:
        if 'h' in time:
            duration = int(time[:-1])*60
        else:
            duration= int(time[:-1])
            
    return duration


def create_preprocess_date_time(df):
    '''
    This Function preprocess date_of_journey and duration to create departure and arrival date time.
    '''
    df['DepartureDateTime'] = df['Date_of_Journey'] + " "+ df['Dep_Time']
    df['DepartureDateTime'] = pd.to_datetime(df['DepartureDateTime'],infer_datetime_format=True)
    df['Duration_min'] = df['Duration'].apply(lambda x: convert_duration_to_minutes(x))
    df['Duration_timedelta'] = pd.to_timedelta(df['Duration_min'], unit='m')
    df["ArrivalDateTime"] = df['DepartureDateTime'] + df['Duration_timedelta']
    return df

stops_dict = {
    'non-stop':0,
    '2 stops':2,
    '1 stop':1,
    '3 stops':3,
    '4 stops':4
}

## Running the process of prediction is also referred as inference:
def sanity_check(df,train=True):
    '''
    This function performs sanity check on the airline data.
    inputs:
        df: dataframe that we need to perform sanity check
        train: This is used for process of training and inference.
            train is having default value of True and can be set as False if we are running inference.
            
        ## process for training    
        sanity_check(df)
        # process of prediction
        sanity_check(df,train=False)
    returns:
        df
    '''
    if train:
        df.drop_duplicates(inplace=True)
    
    create_preprocess_date_time(df)
    df['Total_Stops'] = df['Total_Stops'].replace(stops_dict)
    df.drop(columns=['Date_of_Journey','Dep_Time','Arrival_Time','Duration','Additional_Info'],axis=1,inplace=True)
    
    return df
    
## process for training    
sanity_check(df)
# process of prediction
sanity_check(df,train=False)

# Missing value Handling

In [18]:
# dropping Nan value as its a single observation.
df.dropna(inplace=True)

#### Actions on missing value

In [20]:
def handle_missing_value(df,train=True):
    """
    This function helps to handle missing value.
    Since for Airline data there is just one missing value we can choose to drop missing value.
    inputs:
         df: dataframe which requires imputation.
         
    returns:
        df
    
    """
    df.dropna(inplace=True)
    return df

handle_missing_value(df)

,Airline,Source,Destination,Route,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime
0,IndiGo,Banglore,New Delhi,BLR → DEL,0.0,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2.0,7662,2019-05-01 05:50:00,445,0 days 07:25:00,2019-05-01 13:15:00
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2.0,13882,2019-06-09 09:25:00,1140,0 days 19:00:00,2019-06-10 04:25:00
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,1.0,6218,2019-05-12 18:05:00,325,0 days 05:25:00,2019-05-12 23:30:00
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,1.0,13302,2019-03-01 16:50:00,285,0 days 04:45:00,2019-03-01 21:35:00
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,CCU → BLR,0.0,4107,2019-04-09 19:55:00,150,0 days 02:30:00,2019-04-09 22:25:00
10679,Air India,Kolkata,Banglore,CCU → BLR,0.0,4145,2019-04-27 20:45:00,155,0 days 02:35:00,2019-04-27 23:20:00
10680,Jet Airways,Banglore,Delhi,BLR → DEL,0.0,7229,2019-04-27 08:20:00,180,0 days 03:00:00,2019-04-27 11:20:00
10681,Vistara,Banglore,New Delhi,BLR → DEL,0.0,12648,2019-03-01 11:30:00,160,0 days 02:40:00,2019-03-01 14:10:00


# Handling Categorical Values

In [23]:
df['Destination'] = df['Destination'].replace({'New Delhi':'Delhi'})

In [24]:
# No order in columns
categorical_cols = ['Airline', 'Source', 'Destination', 'Route']
for col in categorical_cols:
    print("Column :",col," Unique values :",df[col].unique()," and No. of uniques : ",df[col].nunique())
    print("-"*90)

Column : Airline  Unique values : ['IndiGo' 'Air India' 'Jet Airways' 'SpiceJet' 'Multiple carriers' 'GoAir'
 'Vistara' 'Air Asia' 'Vistara Premium economy' 'Jet Airways Business'
 'Multiple carriers Premium economy' 'Trujet']  and No. of uniques :  12
------------------------------------------------------------------------------------------
Column : Source  Unique values : ['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai']  and No. of uniques :  5
------------------------------------------------------------------------------------------
Column : Destination  Unique values : ['Delhi' 'Banglore' 'Cochin' 'Kolkata' 'Hyderabad']  and No. of uniques :  5
------------------------------------------------------------------------------------------
Column : Route  Unique values : ['BLR → DEL' 'CCU → IXR → BBI → BLR' 'DEL → LKO → BOM → COK'
 'CCU → NAG → BLR' 'BLR → NAG → DEL' 'CCU → BLR' 'BLR → BOM → DEL'
 'DEL → BOM → COK' 'DEL → BLR → COK' 'MAA → CCU' 'CCU → BOM → BLR'
 'DEL → AMD → BOM → COK'

In [27]:
col = 'Airline'

In [30]:
df.groupby(col).size()

Airline
Air Asia                              319
Air India                            1694
GoAir                                 194
IndiGo                               2043
Jet Airways                          3700
Jet Airways Business                    6
Multiple carriers                    1196
Multiple carriers Premium economy      13
SpiceJet                              815
Trujet                                  1
Vistara                               478
Vistara Premium economy                 3
dtype: int64

In [28]:
freq_value = df.groupby(col).size()/len(df)

In [29]:
freq_value

Airline
Air Asia                             0.030491
Air India                            0.161919
GoAir                                0.018543
IndiGo                               0.195278
Jet Airways                          0.353661
Jet Airways Business                 0.000574
Multiple carriers                    0.114318
Multiple carriers Premium economy    0.001243
SpiceJet                             0.077901
Trujet                               0.000096
Vistara                              0.045689
Vistara Premium economy              0.000287
dtype: float64

In [31]:
freq_dict = freq_value.to_dict()

In [32]:
freq_dict

{'Air Asia': 0.030491301854329956,
 'Air India': 0.1619193270885108,
 'GoAir': 0.018543299560313517,
 'IndiGo': 0.1952781494934047,
 'Jet Airways': 0.35366086790288664,
 'Jet Airways Business': 0.0005735041101127892,
 'Multiple carriers': 0.1143184859491493,
 'Multiple carriers Premium economy': 0.0012425922385777097,
 'SpiceJet': 0.0779009749569872,
 'Trujet': 9.558401835213153e-05,
 'Vistara': 0.04568916077231887,
 'Vistara Premium economy': 0.0002867520550563946}

In [33]:
df["Freq_encoded_"+col] = df[col].replace(freq_dict)

In [34]:
df.head(1)

,Airline,Source,Destination,Route,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime,Freq_encoded_Airline
0,IndiGo,Banglore,Delhi,BLR → DEL,0.0,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00,0.195278


In [46]:
def frequency_encoder(df,col):
    """
    This function encodes a categorical column based on the frequency of their occurence.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          frequency encoded dictionary for columns
    """
    freq_value = df.groupby(col).size()/len(df)
    freq_dict = freq_value.to_dict()
    df["Freq_encoded_"+col] = df[col].replace(freq_dict)
    return freq_dict

In [36]:
col = 'Airline'
target_col = 'Price'

In [37]:
mean_value = df.groupby(col)[target_col].mean()

In [38]:
mean_value

Airline
Air Asia                              5590.260188
Air India                             9556.608028
GoAir                                 5861.056701
IndiGo                                5668.469897
Jet Airways                          11599.021081
Jet Airways Business                 58358.666667
Multiple carriers                    10902.678094
Multiple carriers Premium economy    11418.846154
SpiceJet                              4335.841718
Trujet                                4140.000000
Vistara                               7801.355649
Vistara Premium economy               8962.333333
Name: Price, dtype: float64

In [39]:
mean_dict = mean_value.to_dict()

In [40]:
mean_dict

{'Air Asia': 5590.260188087775,
 'Air India': 9556.6080283353,
 'GoAir': 5861.056701030928,
 'IndiGo': 5668.469897209985,
 'Jet Airways': 11599.02108108108,
 'Jet Airways Business': 58358.666666666664,
 'Multiple carriers': 10902.678093645485,
 'Multiple carriers Premium economy': 11418.846153846154,
 'SpiceJet': 4335.841717791411,
 'Trujet': 4140.0,
 'Vistara': 7801.355648535565,
 'Vistara Premium economy': 8962.333333333334}

In [41]:
df["Mean_encoded_"+col] = df[col].replace(mean_dict)

In [42]:
df.head(1)

,Airline,Source,Destination,Route,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime,Freq_encoded_Airline,Mean_encoded_Airline
0,IndiGo,Banglore,Delhi,BLR → DEL,0.0,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00,0.195278,5668.469897


In [45]:
def mean_encoder(df,col,target_col):
    """
    This function encodes a categorical column based on the frequency of their occurence.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          Mean encoded dict for column
    """
    mean_value = df.groupby(col)[target_col].mean()
    mean_dict = mean_value.to_dict()
    df["Mean_encoded_"+col] = df[col].replace(mean_dict)
    return mean_dict

In [47]:
## Label encoder for function and later usages:

## Applying Label Encoder to Categorical columns as Hit and Trail.
from sklearn.preprocessing import LabelEncoder

def label_encoder(df,col):
    """
    This function encodes a categorical column based on the basis of their order label.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          label encoded dict for column
    """
    le = LabelEncoder()
    le.fit(df[col])
    label_dict = dict(zip((le.classes_),le.transform(le.classes_)))
    df["Label_encoded_"+col] = df[col].replace(label_dict)
    return label_dict

In [48]:
## Create a function to handle categorical value
def handle_categorical_values(df,target):
    '''
      This function handles categorical value and create a dataframe.
      Input:
        df : Dataframe which require categorical value treatment
      returns :
         Dataframe with all categorical value handled.
    '''
    encoded_dict = {}
    # Getting all object columns
    object_columns = df.select_dtypes(object).columns

    ## generate frequency encoded categorical values
    frequency_encoded_dict ={} 
    for col in object_columns:
        freq_dict = frequency_encoder(df,col)
        frequency_encoded_dict[col] = freq_dict

    ## generate target mean encoded categorical values
    mean_encoded_dict ={} 
    for col in object_columns:
        mean_dict = mean_encoder(df,col,target)
        mean_encoded_dict[col] = mean_dict

    
    ## generate label encoded categorical values
    label_encoded_dict ={} 
    for col in object_columns:
        label_dict = label_encoder(df,col)
        label_encoded_dict[col] = label_dict
    
    encoded_dict["Frequency"] = frequency_encoded_dict
    encoded_dict["Mean"] = mean_encoded_dict
    encoded_dict["Label"] = label_encoded_dict

    return df, encoded_dict

In [49]:
df.select_dtypes(object).columns

Index(['Airline', 'Source', 'Destination', 'Route'], dtype='object')

In [50]:
target = 'Price'
df, encoded_dict = handle_categorical_values(df,target)

In [51]:
encoded_dict

{'Frequency': {'Airline': {'Air Asia': 0.030491301854329956,
   'Air India': 0.1619193270885108,
   'GoAir': 0.018543299560313517,
   'IndiGo': 0.1952781494934047,
   'Jet Airways': 0.35366086790288664,
   'Jet Airways Business': 0.0005735041101127892,
   'Multiple carriers': 0.1143184859491493,
   'Multiple carriers Premium economy': 0.0012425922385777097,
   'SpiceJet': 0.0779009749569872,
   'Trujet': 9.558401835213153e-05,
   'Vistara': 0.04568916077231887,
   'Vistara Premium economy': 0.0002867520550563946},
  'Source': {'Banglore': 0.20827757598929458,
   'Chennai': 0.03641751099216211,
   'Delhi': 0.41531255974001147,
   'Kolkata': 0.27337029248709616,
   'Mumbai': 0.06662206079143568},
  'Destination': {'Banglore': 0.27337029248709616,
   'Cochin': 0.41531255974001147,
   'Delhi': 0.20827757598929458,
   'Hyderabad': 0.06662206079143568,
   'Kolkata': 0.03641751099216211},
  'Route': {'BLR → AMD → DEL': 0.0017205123303383673,
   'BLR → BBI → DEL': 0.0004779200917606576,
   'BL

In [53]:
df.head(2)

,Airline,Source,Destination,Route,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime,...,Freq_encoded_Source,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Source,Mean_encoded_Destination,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Source,Label_encoded_Destination,Label_encoded_Route
0,IndiGo,Banglore,Delhi,BLR → DEL,0.0,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00,...,0.208278,0.208278,0.146817,8022.872877,8022.872877,5552.235677,3,0,2,18
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2.0,7662,2019-05-01 05:50:00,445,0 days 07:25:00,2019-05-01 13:15:00,...,0.273370,0.273370,0.000574,9143.083566,9143.083566,7369.166667,1,3,0,84


In [51]:
categorical_cols = ['Airline', 'Source', 'Destination', 'Route']
df.drop(columns=categorical_cols,inplace=True)

#### Action on categorical Values

In [59]:
def frequency_encoder(df,col):
    """
    This function encodes a categorical column based on the frequency of their occurence.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          frequency encoded dictionary for columns
    """
    freq_value = df.groupby(col).size()/len(df)
    freq_dict = freq_value.to_dict()
    df["Freq_encoded_"+col] = df[col].replace(freq_dict)
    return freq_dict


def mean_encoder(df,col,target_col):
    """
    This function encodes a categorical column based on the frequency of their occurence.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          Mean encoded dict for column
    """
    mean_value = df.groupby(col)[target_col].mean()
    mean_dict = mean_value.to_dict()
    df["Mean_encoded_"+col] = df[col].replace(mean_dict)
    return mean_dict


from sklearn.preprocessing import LabelEncoder

def label_encoder(df,col):
    """
    This function encodes a categorical column based on the basis of their order label.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          label encoded dict for column
    """
    le = LabelEncoder()
    le.fit(df[col])
    label_dict = dict(zip((le.classes_),le.transform(le.classes_)))
    df["Label_encoded_"+col] = df[col].replace(label_dict)
    return label_dict


## Create a function to handle categorical value
def handle_categorical_values(df,target):
    '''
      This function handles categorical value and create a dataframe.
      Input:
        df : Dataframe which require categorical value treatment
      returns :
         Dataframe with all categorical value handled.
    '''
    encoded_dict = {}
    # Getting all object columns
    object_columns = df.select_dtypes(object).columns

    ## generate frequency encoded categorical values
    frequency_encoded_dict ={} 
    for col in object_columns:
        freq_dict = frequency_encoder(df,col)
        frequency_encoded_dict[col] = freq_dict

    ## generate target mean encoded categorical values
    mean_encoded_dict ={} 
    for col in object_columns:
        mean_dict = mean_encoder(df,col,target)
        mean_encoded_dict[col] = mean_dict

    
    ## generate label encoded categorical values
    label_encoded_dict ={} 
    for col in object_columns:
        label_dict = label_encoder(df,col)
        label_encoded_dict[col] = label_dict
    
    encoded_dict["Frequency"] = frequency_encoded_dict
    encoded_dict["Mean"] = mean_encoded_dict
    encoded_dict["Label"] = label_encoded_dict

    return df, encoded_dict


def airline_handle_categorical_data(df,target):
    df['Destination'] = df['Destination'].replace({'New Delhi':'Delhi'})
    df, encoded_dict = handle_categorical_values(df,target)
    categorical_cols = df.select_dtypes(object).columns
    df.drop(columns=categorical_cols,inplace=True)
    return df, encoded_dict

df, encoded_dict = airline_handle_categorical_data(df,'Price')

## Correlation

In [64]:
df.corr()

,Total_Stops,Price,Duration_min,Freq_encoded_Airline,Mean_encoded_Airline,Freq_encoded_Source,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Source,Mean_encoded_Destination,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Source,Label_encoded_Destination,Label_encoded_Route
Total_Stops,1.000000,0.602013,0.727982,0.211803,0.421327,0.571402,0.571402,-0.205605,0.559200,0.559200,0.806690,-0.194549,0.194203,-0.422656,0.441347
Price,0.602013,1.000000,0.501758,0.371620,0.638674,0.350997,0.350997,0.038912,0.359938,0.359938,0.746275,-0.036491,0.013796,-0.260751,0.154374
Duration_min,0.727982,0.501758,1.000000,0.289457,0.433989,0.400039,0.400039,-0.111065,0.414954,0.414954,0.671871,-0.153618,0.162808,-0.383773,0.273200
Freq_encoded_Airline,0.211803,0.371620,0.289457,1.000000,0.581861,0.038026,0.038026,0.019636,0.056089,0.056089,0.367567,-0.200348,0.055926,-0.107401,-0.065789
Mean_encoded_Airline,0.421327,0.638674,0.433989,0.581861,1.000000,0.258498,0.258498,0.158467,0.257259,0.257259,0.572144,-0.057135,0.047805,-0.183451,0.071608
Freq_encoded_Source,0.571402,0.350997,0.400039,0.038026,0.258498,1.000000,1.000000,0.303115,0.975159,0.975159,0.470332,-0.049748,0.066914,-0.596026,0.612054
Freq_encoded_Destination,0.571402,0.350997,0.400039,0.038026,0.258498,1.000000,1.000000,0.303115,0.975159,0.975159,0.470332,-0.049748,0.066914,-0.596026,0.612054
Freq_encoded_Route,-0.205605,0.038912,-0.111065,0.019636,0.158467,0.303115,0.303115,1.000000,0.272478,0.272478,0.052142,0.194054,-0.196373,-0.001272,0.055678
Mean_encoded_Source,0.559200,0.359938,0.414954,0.056089,0.257259,0.975159,0.975159,0.272478,1.000000,1.000000,0.482313,-0.059864,0.038329,-0.724431,0.496810
Mean_encoded_Destination,0.559200,0.359938,0.414954,0.056089,0.257259,0.975159,0.975159,0.272478,1.000000,1.000000,0.482313,-0.059864,0.038329,-0.724431,0.496810


In [77]:
from sklearn.model_selection import train_test_split

from sklearn import linear_model
from sklearn import metrics

from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import SplineTransformer

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor


def regression(X_train, X_test, y_train, y_test):
    ## Baseline model
    reg = linear_model.LinearRegression()
    reg.fit(X_train,y_train)
    print(metrics.r2_score(y_train,reg.predict(X_train)), metrics.r2_score(y_test,reg.predict(X_test)))
    return reg

def knearestneighbour(X_train, X_test, y_train, y_test):
    knn_regressor = KNeighborsRegressor(n_neighbors=10)
    knn_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,knn_regressor.predict(X_train)),metrics.r2_score(y_test,knn_regressor.predict(X_test)))
    return knn_regressor

def decisiontree(X_train, X_test, y_train, y_test):
    tree = DecisionTreeRegressor(max_depth=10)
    tree.fit(X_train,y_train)
    print(metrics.r2_score(y_train,tree.predict(X_train)),metrics.r2_score(y_test,tree.predict(X_test)))
    return tree

def randomForest(X_train, X_test, y_train, y_test):
    random_regressor = RandomForestRegressor(n_estimators=500, min_samples_split=10,min_samples_leaf=2)
    random_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,random_regressor.predict(X_train)),metrics.r2_score(y_test,random_regressor.predict(X_test)))
    return random_regressor

from xgboost import XGBRegressor
def xgboost(X_train, X_test, y_train, y_test):
    xgboost_regressor = XGBRegressor()
    xgboost_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,xgboost_regressor.predict(X_train)),metrics.r2_score(y_test,xgboost_regressor.predict(X_test)))
    return xgboost_regressor

from catboost import  CatBoostRegressor
def catboost(X_train, X_test, y_train, y_test):
    catboost_regressor = CatBoostRegressor(verbose=False)
    catboost_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,catboost_regressor.predict(X_train)),metrics.r2_score(y_test,catboost_regressor.predict(X_test)))
    return catboost_regressor

from sklearn.ensemble import GradientBoostingRegressor
def gboost(X_train, X_test, y_train, y_test):
    gboost_regressor = GradientBoostingRegressor()
    gboost_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,gboost_regressor.predict(X_train)),metrics.r2_score(y_test,gboost_regressor.predict(X_test)))
    return gboost_regressor


def train(X,y,modelType):
    # Split your dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    model = modelType(X_train, X_test, y_train, y_test)
    return model


In [65]:
df.head(2)

,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime,Freq_encoded_Airline,Mean_encoded_Airline,Freq_encoded_Source,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Source,Mean_encoded_Destination,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Source,Label_encoded_Destination,Label_encoded_Route
0,0.0,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00,0.195278,5668.469897,0.208278,0.208278,0.146817,8022.872877,8022.872877,5552.235677,3,0,2,18
1,2.0,7662,2019-05-01 05:50:00,445,0 days 07:25:00,2019-05-01 13:15:00,0.161919,9556.608028,0.273370,0.273370,0.000574,9143.083566,9143.083566,7369.166667,1,3,0,84


In [71]:
X = df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = df['Price']

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
reg = linear_model.LinearRegression()
reg.fit(X_train,y_train)
print(metrics.r2_score(y_train,reg.predict(X_train)), metrics.r2_score(y_test,reg.predict(X_test)))

0.6334776126351567 0.6191483972302606


In [73]:
train(X,y,regression)

0.6334776126351567 0.6191483972302606


LinearRegression()

In [76]:
train(X,y,knearestneighbour)

0.6891587105979624 0.6326587229158694


KNeighborsRegressor(n_neighbors=10)

In [78]:
train(X,y,decisiontree)

0.7132738528455688 0.6307262225774899


DecisionTreeRegressor(max_depth=10)

In [79]:
train(X,y,randomForest)

0.7357941829294776 0.6412501378086872


RandomForestRegressor(min_samples_leaf=2, min_samples_split=10,
                      n_estimators=500)

In [80]:
train(X,y,xgboost)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.7522506912631617 0.6270171749071364


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [81]:
train(X,y,catboost)

0.7290013853336741 0.6484805676829498


In [82]:
train(X,y,gboost)

0.6835166496109661 0.6465657512106835


GradientBoostingRegressor()

In [ ]:
## Convert all data into numerical form to obtain your baseline model.

In [83]:
from sklearn.linear_model import LinearRegression

def sklearn_vif(exogs, data):
    '''
    This function calculates variance inflation function in sklearn way. 
     It is a comparatively faster process.
    '''
    # initialize dictionaries
    vif_dict, tolerance_dict = {}, {}

    # form input data for each exogenous variable
    for exog in exogs:
        not_exog = [i for i in exogs if i != exog]
        X, y = data[not_exog], data[exog]

        # extract r-squared from the fit
        r_squared = LinearRegression().fit(X, y).score(X, y)

        # calculate VIF
        vif = 1/(1 - r_squared)
        vif_dict[exog] = vif

        # calculate tolerance
        tolerance = 1 - r_squared
        tolerance_dict[exog] = tolerance

    # return VIF DataFrame
    df_vif = pd.DataFrame({'VIF': vif_dict, 'Tolerance': tolerance_dict})

    return df_vif

df_vif= sklearn_vif(exogs=X.columns, data=X).sort_values(by='VIF',ascending=False)

C:\Users\theve\AppData\Local\Temp/ipykernel_15328/2639988759.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_15328/2639988759.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_15328/2639988759.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_15328/2639988759.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)


In [84]:
# VIF >= 10 shows high collinearity
df_vif

,VIF,Tolerance
Freq_encoded_Source,inf,0.000000
Freq_encoded_Destination,inf,0.000000
Mean_encoded_Source,inf,0.000000
Mean_encoded_Destination,inf,0.000000
Label_encoded_Destination,6.550249e+04,0.000015
Label_encoded_Source,1.462523e+04,0.000068
Label_encoded_Route,5.382358e+01,0.018579
Total_Stops,8.105551e+00,0.123372
Mean_encoded_Route,6.026228e+00,0.165941
Duration_min,2.349205e+00,0.425676


In [85]:
X.columns

Index(['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 'Freq_encoded_Source',
       'Freq_encoded_Destination', 'Freq_encoded_Route', 'Mean_encoded_Source',
       'Mean_encoded_Destination', 'Mean_encoded_Route',
       'Label_encoded_Airline', 'Label_encoded_Source',
       'Label_encoded_Destination', 'Label_encoded_Route'],
      dtype='object')

In [90]:
vif_cols = ['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 
#             'Freq_encoded_Source',
       'Freq_encoded_Destination', 'Freq_encoded_Route', 
#             'Mean_encoded_Source',
#        'Mean_encoded_Destination',
            'Mean_encoded_Route',
       'Label_encoded_Airline', 'Label_encoded_Source',
       'Label_encoded_Destination', 'Label_encoded_Route']
X[vif_cols]
df_vif= sklearn_vif(exogs=vif_cols, data=X[vif_cols]).sort_values(by='VIF',ascending=False)
df_vif

,VIF,Tolerance
Total_Stops,6.936320,0.144169
Freq_encoded_Destination,5.917626,0.168987
Mean_encoded_Route,4.841500,0.206548
Label_encoded_Route,3.320321,0.301176
Label_encoded_Destination,3.302453,0.302805
Label_encoded_Source,2.452627,0.407726
Duration_min,2.348979,0.425717
Mean_encoded_Airline,2.072872,0.482423
Freq_encoded_Route,1.940591,0.515307
Freq_encoded_Airline,1.704650,0.586631


In [140]:
train(X[vif_cols],y,regression)
train(X[vif_cols],y,knearestneighbour)
train(X[vif_cols],y,decisiontree)
train(X[vif_cols],y,randomForest)
train(X[vif_cols],y,xgboost)
train(X[vif_cols],y,gboost)
train(X[vif_cols],y,catboost)

0.6332966514192129 0.6190857520128911
0.6901263709827972 0.6322774825953947
0.7132738528455688 0.6327580377742608
0.7359922062542179 0.642213986434599


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.7522506912631617 0.6270171749071364
0.6835166496109661 0.6465457537974091
0.7294873492152483 0.6485187367194287


## Scaling and Outlier treatment

In [97]:
X = X[vif_cols]

In [99]:
from sklearn.preprocessing import RobustScaler

X_transform = RobustScaler().fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=X.columns)

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)
train(X_transformed,y,randomForest)
train(X_transformed,y,xgboost)
train(X_transformed,y,gboost)
train(X_transformed,y,catboost)

0.6332966514192129 0.6190857520128925
0.6833931651326812 0.6298425588036669
0.7132738528455688 0.6341841859869738
0.7357314414333066 0.642741999408352


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.7522506912631617 0.6270169741947662
0.6835166496109661 0.64731177803766
0.7294873492152483 0.6485187099733847


In [100]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

,Total_Stops,Duration_min,Freq_encoded_Airline,Mean_encoded_Airline,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Source,Label_encoded_Destination,Label_encoded_Route,Price
Total_Stops,1.000000,0.727982,0.211803,0.421327,0.571402,-0.205605,0.806690,-0.194549,0.194203,-0.422656,0.441347,0.039017
Duration_min,0.727982,1.000000,0.289457,0.433989,0.400039,-0.111065,0.671871,-0.153618,0.162808,-0.383773,0.273200,0.027030
Freq_encoded_Airline,0.211803,0.289457,1.000000,0.581861,0.038026,0.019636,0.367567,-0.200348,0.055926,-0.107401,-0.065789,0.006926
Mean_encoded_Airline,0.421327,0.433989,0.581861,1.000000,0.258498,0.158467,0.572144,-0.057135,0.047805,-0.183451,0.071608,0.049989
Freq_encoded_Destination,0.571402,0.400039,0.038026,0.258498,1.000000,0.303115,0.470332,-0.049748,0.066914,-0.596026,0.612054,0.039077
Freq_encoded_Route,-0.205605,-0.111065,0.019636,0.158467,0.303115,1.000000,0.052142,0.194054,-0.196373,-0.001272,0.055678,0.027073
Mean_encoded_Route,0.806690,0.671871,0.367567,0.572144,0.470332,0.052142,1.000000,-0.058168,0.018486,-0.349403,0.206860,0.054850
Label_encoded_Airline,-0.194549,-0.153618,-0.200348,-0.057135,-0.049748,0.194054,-0.058168,1.000000,-0.012172,0.068634,0.029636,0.011536
Label_encoded_Source,0.194203,0.162808,0.055926,0.047805,0.066914,-0.196373,0.018486,-0.012172,1.000000,-0.432609,0.403884,0.001868
Label_encoded_Destination,-0.422656,-0.383773,-0.107401,-0.183451,-0.596026,-0.001272,-0.349403,0.068634,-0.432609,1.000000,-0.229487,-0.023831


In [101]:
from sklearn.preprocessing import PowerTransformer

X_transform = PowerTransformer().fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=X.columns)

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)
train(X_transformed,y,randomForest)
train(X_transformed,y,xgboost)
train(X_transformed,y,gboost)
train(X_transformed,y,catboost)

0.6161029821417293 0.608702206383129
0.6859186526703424 0.6350464683304622
0.7132738528455688 0.632297306873971
0.7355383368934032 0.6434763273576803


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.7522506912631617 0.6270078817520697
0.6835166496109661 0.6469902222088109
0.7294873492152483 0.6486037627372357


In [102]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

,Total_Stops,Duration_min,Freq_encoded_Airline,Mean_encoded_Airline,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Source,Label_encoded_Destination,Label_encoded_Route,Price
Total_Stops,1.000000,0.844850,0.226303,0.466701,0.594835,-0.257056,0.844533,-0.154681,0.178948,-0.373699,0.458192,0.039913
Duration_min,0.844850,1.000000,0.291936,0.524692,0.535225,-0.086790,0.819401,-0.108921,0.103656,-0.425656,0.352314,0.036260
Freq_encoded_Airline,0.226303,0.291936,1.000000,0.600581,0.025808,0.029916,0.355888,-0.081844,0.058761,-0.097448,-0.060665,0.006537
Mean_encoded_Airline,0.466701,0.524692,0.600581,1.000000,0.267096,0.140361,0.601058,-0.014827,0.044355,-0.139421,0.081027,0.045154
Freq_encoded_Destination,0.594835,0.535225,0.025808,0.267096,1.000000,0.215073,0.490481,-0.015420,0.040707,-0.328968,0.687665,0.038186
Freq_encoded_Route,-0.257056,-0.086790,0.029916,0.140361,0.215073,1.000000,0.009358,0.251402,-0.206968,0.087376,-0.032610,0.022537
Mean_encoded_Route,0.844533,0.819401,0.355888,0.601058,0.490481,0.009358,1.000000,-0.017892,-0.005291,-0.296602,0.239712,0.054375
Label_encoded_Airline,-0.154681,-0.108921,-0.081844,-0.014827,-0.015420,0.251402,-0.017892,1.000000,-0.010674,0.054979,0.042487,0.013280
Label_encoded_Source,0.178948,0.103656,0.058761,0.044355,0.040707,-0.206968,-0.005291,-0.010674,1.000000,-0.508930,0.342480,0.000447
Label_encoded_Destination,-0.373699,-0.425656,-0.097448,-0.139421,-0.328968,0.087376,-0.296602,0.054979,-0.508930,1.000000,-0.191226,-0.015926


In [103]:
from sklearn.preprocessing import FunctionTransformer

X_transform = FunctionTransformer(np.log1p).fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=X.columns)


train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)
train(X_transformed,y,randomForest)
train(X_transformed,y,xgboost)
train(X_transformed,y,gboost)
train(X_transformed,y,catboost)

0.5927171308555409 0.5958671491344738
0.6699874172870499 0.6274329302906541
0.7132738528455688 0.6354586666815691
0.736049035689008 0.6425353000957195


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.7522506912631617 0.6270171749071364
0.6835166496109661 0.6466713431372284
0.7294873492152483 0.6486040573582539


In [104]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

,Total_Stops,Duration_min,Freq_encoded_Airline,Mean_encoded_Airline,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Source,Label_encoded_Destination,Label_encoded_Route,Price
Total_Stops,1.000000,0.863786,0.232432,0.490239,0.615532,-0.146651,0.867788,-0.115757,0.313500,-0.378406,0.368466,0.626180
Duration_min,0.863786,1.000000,0.288951,0.544532,0.584356,-0.018656,0.850498,-0.084262,0.214511,-0.425020,0.273478,0.601974
Freq_encoded_Airline,0.232432,0.288951,1.000000,0.601074,0.044400,0.019247,0.342026,0.035451,0.045841,-0.096738,-0.076925,0.366855
Mean_encoded_Airline,0.490239,0.544532,0.601074,1.000000,0.289999,0.163356,0.609313,0.004491,0.082928,-0.137142,0.040957,0.610871
Freq_encoded_Destination,0.615532,0.584356,0.044400,0.289999,1.000000,0.284566,0.563480,-0.016786,0.219333,-0.431073,0.502693,0.354439
Freq_encoded_Route,-0.146651,-0.018656,0.019247,0.163356,0.284566,1.000000,0.096027,0.257797,-0.150866,0.101184,0.048743,0.029534
Mean_encoded_Route,0.867788,0.850498,0.342026,0.609313,0.563480,0.096027,1.000000,0.013449,0.095484,-0.312079,0.137305,0.734900
Label_encoded_Airline,-0.115757,-0.084262,0.035451,0.004491,-0.016786,0.257797,0.013449,1.000000,0.004276,0.057037,0.030229,0.034959
Label_encoded_Source,0.313500,0.214511,0.045841,0.082928,0.219333,-0.150866,0.095484,0.004276,1.000000,-0.556682,0.738598,0.067974
Label_encoded_Destination,-0.378406,-0.425020,-0.096738,-0.137142,-0.431073,0.101184,-0.312079,0.057037,-0.556682,1.000000,-0.341735,-0.195564


In [105]:
from sklearn.preprocessing import FunctionTransformer

def cube(X): return X**(0.333)

X_transform = FunctionTransformer(cube).fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=X.columns)


train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)
train(X_transformed,y,randomForest)
train(X_transformed,y,xgboost)
train(X_transformed,y,gboost)
train(X_transformed,y,catboost)

0.6058911493540655 0.6030285078356926
0.6862846120786659 0.6362939130996079
0.7132738528455688 0.6354588775445703
0.7358178824138835 0.6423133564713615


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.7522506912631617 0.6270192418122551
0.6835166496109661 0.6465766222786796
0.7294873492152483 0.6486040573582539


In [106]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

,Total_Stops,Duration_min,Freq_encoded_Airline,Mean_encoded_Airline,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Source,Label_encoded_Destination,Label_encoded_Route,Price
Total_Stops,1.000000,0.833065,0.239045,0.495621,0.621873,-0.232358,0.873024,-0.043271,0.363901,-0.273600,0.420080,0.628457
Duration_min,0.833065,1.000000,0.286370,0.527382,0.526672,-0.176549,0.799758,-0.067232,0.256528,-0.294052,0.297467,0.576777
Freq_encoded_Airline,0.239045,0.286370,1.000000,0.562445,0.063097,0.027772,0.328025,0.077002,0.049393,-0.072750,-0.035205,0.333373
Mean_encoded_Airline,0.495621,0.527382,0.562445,1.000000,0.290668,0.107632,0.607769,0.059308,0.086410,-0.069334,0.060645,0.621726
Freq_encoded_Destination,0.621873,0.526672,0.063097,0.290668,1.000000,0.142225,0.541300,-0.036344,0.205377,-0.286197,0.418825,0.358264
Freq_encoded_Route,-0.232358,-0.176549,0.027772,0.107632,0.142225,1.000000,-0.058498,0.261455,-0.135131,0.135966,-0.040536,-0.067720
Mean_encoded_Route,0.873024,0.799758,0.328025,0.607769,0.541300,-0.058498,1.000000,0.034782,0.128585,-0.182592,0.175625,0.740985
Label_encoded_Airline,-0.043271,-0.067232,0.077002,0.059308,-0.036344,0.261455,0.034782,1.000000,0.016958,0.065459,0.039893,0.054084
Label_encoded_Source,0.363901,0.256528,0.049393,0.086410,0.205377,-0.135131,0.128585,0.016958,1.000000,-0.497611,0.790576,0.086664
Label_encoded_Destination,-0.273600,-0.294052,-0.072750,-0.069334,-0.286197,0.135966,-0.182592,0.065459,-0.497611,1.000000,-0.200193,-0.118911


In [107]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False,interaction_only=False)

X_transform = poly.fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=poly.get_feature_names(X.columns))

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)
train(X_transformed,y,randomForest)
train(X_transformed,y,xgboost)
train(X_transformed,y,gboost)
train(X_transformed,y,catboost)

C:\Users\theve\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


0.66565218787968 0.6466279806420066
0.693267321244746 0.6286954930586043
0.7181320979275021 0.6165351652049093
0.7402423587894273 0.6428980548148997


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.7551481951752836 0.6230385352017183
0.6984709434592705 0.6533653763250008
0.7394861704394259 0.6451683132278625


In [109]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

,Total_Stops,Duration_min,Freq_encoded_Airline,Mean_encoded_Airline,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Source,Label_encoded_Destination,...,Label_encoded_Airline Label_encoded_Source,Label_encoded_Airline Label_encoded_Destination,Label_encoded_Airline Label_encoded_Route,Label_encoded_Source^2,Label_encoded_Source Label_encoded_Destination,Label_encoded_Source Label_encoded_Route,Label_encoded_Destination^2,Label_encoded_Destination Label_encoded_Route,Label_encoded_Route^2,Price
Total_Stops,1.000000,0.727982,0.211803,0.421327,0.571402,-0.205605,0.806690,-0.194549,0.194203,-0.422656,...,0.006334,-0.374143,0.113162,0.012053,-0.166049,0.461468,-0.464628,-0.124019,0.424676,0.039017
Duration_min,0.727982,1.000000,0.289457,0.433989,0.400039,-0.111065,0.671871,-0.153618,0.162808,-0.383773,...,0.027786,-0.341376,0.026229,0.033718,-0.193404,0.325861,-0.389575,-0.165581,0.247025,0.027030
Freq_encoded_Airline,0.211803,0.289457,1.000000,0.581861,0.038026,0.019636,0.367567,-0.200348,0.055926,-0.107401,...,-0.086083,-0.206214,-0.188971,0.062171,-0.037090,0.020606,-0.113188,-0.147209,-0.075729,0.006926
Mean_encoded_Airline,0.421327,0.433989,0.581861,1.000000,0.258498,0.158467,0.572144,-0.057135,0.047805,-0.183451,...,-0.019807,-0.184870,0.026962,-0.007029,-0.069993,0.116928,-0.243205,-0.157479,0.061268,0.049989
Freq_encoded_Destination,0.571402,0.400039,0.038026,0.258498,1.000000,0.303115,0.470332,-0.049748,0.066914,-0.596026,...,0.022141,-0.467312,0.304091,-0.225234,-0.356934,0.509746,-0.757440,-0.201650,0.593103,0.039077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Label_encoded_Source Label_encoded_Route,0.461468,0.325861,0.020606,0.116928,0.509746,-0.081101,0.217280,-0.009361,0.832448,-0.525916,...,0.537603,-0.352556,0.467304,0.600611,0.234018,1.000000,-0.388180,0.099221,0.674208,0.015284
Label_encoded_Destination^2,-0.464628,-0.389575,-0.113188,-0.243205,-0.757440,-0.140682,-0.415996,0.090632,-0.222891,0.924745,...,-0.108130,0.735887,0.014021,-0.056308,0.578380,-0.388180,1.000000,0.752854,-0.053974,-0.032532
Label_encoded_Destination Label_encoded_Route,-0.124019,-0.165581,-0.147209,-0.157479,-0.201650,-0.019444,-0.204262,0.101591,-0.075688,0.672339,...,-0.012108,0.587980,0.423943,-0.151155,0.476360,0.099221,0.752854,1.000000,0.603861,-0.016417
Label_encoded_Route^2,0.424676,0.247025,-0.075729,0.061268,0.593103,0.057786,0.214802,0.035101,0.211486,-0.082253,...,0.144266,0.006069,0.622013,-0.092702,0.101252,0.674208,-0.053974,0.603861,1.000000,0.013501


## Adding Basic Time based Features

## I

In [111]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)

            # 3-8 early_morning
            # 8-12 morning
            # 12-16 afternoon
            # 16-20 evening
            # 20-00 night
            # 00-3 late_night
            

In [119]:
df['dep_hr'] = df['DepartureDateTime'].dt.hour
df['arr_hr'] = df['ArrivalDateTime'].dt.hour

df['departure_timeOfDay'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
df['arrival_timeOfDay'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

In [123]:
X = df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = df['Price']

In [125]:
train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)
train(X_transformed,y,randomForest)
train(X_transformed,y,xgboost)
train(X_transformed,y,gboost)
train(X_transformed,y,catboost)

0.66565218787968 0.6466279806420066
0.693267321244746 0.6286954930586043
0.7181320979275021 0.6161129730579327
0.7400087681709059 0.6435515995264016


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.7551481951752836 0.6230385352017183
0.6984709434592706 0.6533055776055843
0.7394861704394259 0.6451683132278625


## II

In [128]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)

        # 3-8 early_morning
        # 8-12 morning
        # 12-16 afternoon
        # 16-20 evening
        # 20-00 night
        # 00-3 late_night
        
df['dep_hr'] = df['DepartureDateTime'].dt.hour
df['arr_hr'] = df['ArrivalDateTime'].dt.hour

df['departure_timeOfDay'] = df['dep_hr'].apply(lambda x: time_of_day(x)[0])
df['arrival_timeOfDay'] = df['arr_hr'].apply(lambda x: time_of_day(x)[0])

df['departure_timeOfDay_encoded'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
df['arrival_timeOfDay_encoded'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

one_hot_cols = ['departure_timeOfDay','arrival_timeOfDay']
df_oneHotEncoded = pd.get_dummies(df[one_hot_cols])

new_df = pd.concat([df,df_oneHotEncoded],axis=1)

new_df.drop(columns=one_hot_cols,inplace=True)

In [129]:
X = new_df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = new_df['Price']

print(X.columns)

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)

model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,gboost)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

model = train(X,y,catboost)
pd.DataFrame({'feature_names_in_':X.columns,'feature_importances_':model.get_feature_importance()}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

Index(['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 'Freq_encoded_Source',
       'Freq_encoded_Destination', 'Freq_encoded_Route', 'Mean_encoded_Source',
       'Mean_encoded_Destination', 'Mean_encoded_Route',
       'Label_encoded_Airline', 'Label_encoded_Source',
       'Label_encoded_Destination', 'Label_encoded_Route', 'dep_hr', 'arr_hr',
       'departure_timeOfDay_encoded', 'arrival_timeOfDay_encoded',
       'departure_timeOfDay_after_noon', 'departure_timeOfDay_early_morning',
       'departure_timeOfDay_evening', 'departure_timeOfDay_late_night',
       'departure_timeOfDay_morning', 'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening', 'arrival_timeOfDay_late_night',
       'arrival_timeOfDay_morning', 'arrival_timeOfDay_night'],
      dtype='object')
0.66565218787968 0.6466279806420066
0.693267321244746 0.6286954930586043
0.7181320979275021 0.617121

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.7912406373444628 0.6119174892433836
0.7678474388048799 0.6347444417199405
0.7684114889306505 0.6376201723732093


,feature_names_in_,feature_importances_
9,Mean_encoded_Route,0.662326
3,Mean_encoded_Airline,0.099174
1,Duration_min,0.088206
2,Freq_encoded_Airline,0.042925
14,dep_hr,0.026104
15,arr_hr,0.024753
6,Freq_encoded_Route,0.007694
17,arrival_timeOfDay_encoded,0.005567
10,Label_encoded_Airline,0.005211
13,Label_encoded_Route,0.004597


## III

In [130]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)

# 3-8 early_morning
# 8-12 morning
# 12-16 afternoon
# 16-20 evening
# 20-00 night
# 00-3 late_night

df['dep_hr'] = df['DepartureDateTime'].dt.hour
df['arr_hr'] = df['ArrivalDateTime'].dt.hour

df['dep_day_of_week'] = df['DepartureDateTime'].dt.day_of_week 
df['arr_day_of_week'] = df['ArrivalDateTime'].dt.day_of_week 

df['dep_weekday'] = np.where(df["dep_day_of_week"].isin([5,6]),0,1)
df['arr_weekday'] = np.where(df["arr_day_of_week"].isin([5,6]),0,1)


df['departure_timeOfDay'] = df['dep_hr'].apply(lambda x: time_of_day(x)[0])
df['departure_timeOfDay_encoded'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
df['arrival_timeOfDay'] = df['arr_hr'].apply(lambda x: time_of_day(x)[0])
df['arrival_timeOfDay_encoded'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

one_hot_cols = ['departure_timeOfDay','arrival_timeOfDay']
df_oneHotEncoded = pd.get_dummies(df[one_hot_cols])
new_df = pd.concat([df,df_oneHotEncoded],axis=1)
new_df.drop(columns=one_hot_cols,inplace=True)

In [ ]:
X = new_df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = new_df['Price']

print(X.columns)

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)

In [131]:
X = new_df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = new_df['Price']

print(X.columns)

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)

model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,gboost)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

model = train(X,y,catboost)
pd.DataFrame({'feature_names_in_':X.columns,'feature_importances_':model.get_feature_importance()}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

Index(['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 'Freq_encoded_Source',
       'Freq_encoded_Destination', 'Freq_encoded_Route', 'Mean_encoded_Source',
       'Mean_encoded_Destination', 'Mean_encoded_Route',
       'Label_encoded_Airline', 'Label_encoded_Source',
       'Label_encoded_Destination', 'Label_encoded_Route', 'dep_hr', 'arr_hr',
       'departure_timeOfDay_encoded', 'arrival_timeOfDay_encoded',
       'dep_day_of_week', 'arr_day_of_week', 'dep_weekday', 'arr_weekday',
       'departure_timeOfDay_after_noon', 'departure_timeOfDay_early_morning',
       'departure_timeOfDay_evening', 'departure_timeOfDay_late_night',
       'departure_timeOfDay_morning', 'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening', 'arrival_timeOfDay_late_night',
       'arrival_timeOfDay_morning', 'arrival_timeOfDay_night'],
      dtype='object')
0.66565218787968 0.646627

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.8731904405581098 0.6951946870386574
0.839683259232454 0.7187248150027368
0.8354910958716206 0.6974194976642514


,feature_names_in_,feature_importances_
9,Mean_encoded_Route,0.616704
3,Mean_encoded_Airline,0.091122
1,Duration_min,0.070834
18,dep_day_of_week,0.050233
2,Freq_encoded_Airline,0.041018
14,dep_hr,0.022794
15,arr_hr,0.020210
20,dep_weekday,0.011288
19,arr_day_of_week,0.010948
6,Freq_encoded_Route,0.008628


## IV

In [132]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)
# 3-8 early_morning
# 8-12 morning
# 12-16 afternoon
# 16-20 evening
# 20-00 night
# 00-3 late_night



df['dep_hr'] = df['DepartureDateTime'].dt.hour
df['arr_hr'] = df['ArrivalDateTime'].dt.hour

df['dep_month'] = df['DepartureDateTime'].dt.month
df['dep_day_of_month'] = df['DepartureDateTime'].dt.day

df['arr_month'] = df['ArrivalDateTime'].dt.month
df['arr_day_of_month'] = df['ArrivalDateTime'].dt.day

df['dep_day_of_week'] = df['DepartureDateTime'].dt.day_of_week 
df['arr_day_of_week'] = df['ArrivalDateTime'].dt.day_of_week 

df['dep_weekday'] = np.where(df["dep_day_of_week"].isin([5,6]),0,1)
df['arr_weekday'] = np.where(df["arr_day_of_week"].isin([5,6]),0,1)


df['departure_timeOfDay'] = df['dep_hr'].apply(lambda x: time_of_day(x)[0])
df['departure_timeOfDay_encoded'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
df['arrival_timeOfDay'] = df['arr_hr'].apply(lambda x: time_of_day(x)[0])
df['arrival_timeOfDay_encoded'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

one_hot_cols = ['departure_timeOfDay','arrival_timeOfDay']
df_oneHotEncoded = pd.get_dummies(df[one_hot_cols])
new_df = pd.concat([df,df_oneHotEncoded],axis=1)
new_df.drop(columns=one_hot_cols,inplace=True)

In [133]:
X = new_df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = new_df['Price']

print(X.columns)

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)


model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,gboost)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

model = train(X,y,catboost)
pd.DataFrame({'feature_names_in_':X.columns,'feature_importances_':model.get_feature_importance()}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

Index(['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 'Freq_encoded_Source',
       'Freq_encoded_Destination', 'Freq_encoded_Route', 'Mean_encoded_Source',
       'Mean_encoded_Destination', 'Mean_encoded_Route',
       'Label_encoded_Airline', 'Label_encoded_Source',
       'Label_encoded_Destination', 'Label_encoded_Route', 'dep_hr', 'arr_hr',
       'departure_timeOfDay_encoded', 'arrival_timeOfDay_encoded',
       'dep_day_of_week', 'arr_day_of_week', 'dep_weekday', 'arr_weekday',
       'dep_month', 'dep_day_of_month', 'arr_month', 'arr_day_of_month',
       'departure_timeOfDay_after_noon', 'departure_timeOfDay_early_morning',
       'departure_timeOfDay_evening', 'departure_timeOfDay_late_night',
       'departure_timeOfDay_morning', 'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening', 'arrival_timeOfDay_late_night',
       'arrival_timeOfDay_morning', 'a

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.9475139066284288 0.84177489377797
0.9227786238229893 0.8495072653052934
0.9132112252698652 0.8349797111028442


,feature_names_in_,feature_importances_
9,Mean_encoded_Route,0.577768
3,Mean_encoded_Airline,0.083447
1,Duration_min,0.049019
23,dep_day_of_month,0.046874
25,arr_day_of_month,0.046864
2,Freq_encoded_Airline,0.039089
24,arr_month,0.025060
22,dep_month,0.024060
13,Label_encoded_Route,0.018058
14,dep_hr,0.013251


In [134]:
X = new_df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = new_df['Price']

print(X.columns)

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)

Index(['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 'Freq_encoded_Source',
       'Freq_encoded_Destination', 'Freq_encoded_Route', 'Mean_encoded_Source',
       'Mean_encoded_Destination', 'Mean_encoded_Route',
       'Label_encoded_Airline', 'Label_encoded_Source',
       'Label_encoded_Destination', 'Label_encoded_Route', 'dep_hr', 'arr_hr',
       'departure_timeOfDay_encoded', 'arrival_timeOfDay_encoded',
       'dep_day_of_week', 'arr_day_of_week', 'dep_weekday', 'arr_weekday',
       'dep_month', 'dep_day_of_month', 'arr_month', 'arr_day_of_month',
       'departure_timeOfDay_after_noon', 'departure_timeOfDay_early_morning',
       'departure_timeOfDay_evening', 'departure_timeOfDay_late_night',
       'departure_timeOfDay_morning', 'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening', 'arrival_timeOfDay_late_night',
       'arrival_timeOfDay_morning', 'a

DecisionTreeRegressor(max_depth=10)

In [135]:
model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

0.9128191545993263 0.8355581450146231


,feature_names_in_,feature_importances_
9,Mean_encoded_Route,0.579522
3,Mean_encoded_Airline,0.083724
1,Duration_min,0.049157
23,dep_day_of_month,0.048292
25,arr_day_of_month,0.045274
2,Freq_encoded_Airline,0.037195
24,arr_month,0.024860
22,dep_month,0.023936
13,Label_encoded_Route,0.018607
14,dep_hr,0.013294


In [136]:
model = train(X,y,gboost)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

0.8287453559771741 0.8011326266295516


,feature_names_in_,feature_importances_
9,Mean_encoded_Route,0.602079
3,Mean_encoded_Airline,0.141283
25,arr_day_of_month,0.047480
23,dep_day_of_month,0.039085
22,dep_month,0.038873
24,arr_month,0.028280
1,Duration_min,0.021851
2,Freq_encoded_Airline,0.019074
15,arr_hr,0.015338
6,Freq_encoded_Route,0.011862


In [137]:
model = train(X,y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.9475139066284288 0.84177489377797


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


,weight,gain,total_gain,cover,total_cover
Total_Stops,80.0,1.356959e+07,1.085567e+09,335.412506,26833.0
Duration_min,1048.0,1.503281e+07,1.575439e+10,1173.386475,1229709.0
Freq_encoded_Airline,211.0,8.203056e+07,1.730845e+10,862.815186,182054.0
Mean_encoded_Airline,121.0,1.172841e+08,1.419138e+10,1384.000000,167464.0
Freq_encoded_Source,45.0,1.456858e+07,6.555862e+08,630.577759,28376.0
Freq_encoded_Route,280.0,1.219211e+07,3.413791e+09,914.432129,256041.0
Mean_encoded_Route,265.0,6.017502e+08,1.594638e+11,1321.022583,350071.0
Label_encoded_Airline,90.0,1.818257e+07,1.636431e+09,940.222229,84620.0
Label_encoded_Source,20.0,3.681148e+07,7.362296e+08,801.099976,16022.0
Label_encoded_Destination,18.0,7.722954e+06,1.390132e+08,1208.000000,21744.0


In [138]:
model = train(X,y,catboost)
pd.DataFrame({'feature_names_in_':X.columns,'feature_importances_':model.get_feature_importance()}).sort_values(by='feature_importances_',ascending=False)

0.9227786238229893 0.8495072653052934


,feature_names_in_,feature_importances_
9,Mean_encoded_Route,26.346807
23,dep_day_of_month,9.561542
3,Mean_encoded_Airline,9.236876
1,Duration_min,8.506824
22,dep_month,7.898531
24,arr_month,7.118499
25,arr_day_of_month,5.578773
2,Freq_encoded_Airline,4.141642
6,Freq_encoded_Route,3.242113
15,arr_hr,2.536854


In [139]:
## After doing Time-based feature Engineering we found best model to be xgboost.

In [141]:
model = train(X,y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.9475139066284288 0.84177489377797


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


,weight,gain,total_gain,cover,total_cover
Total_Stops,80.0,1.356959e+07,1.085567e+09,335.412506,26833.0
Duration_min,1048.0,1.503281e+07,1.575439e+10,1173.386475,1229709.0
Freq_encoded_Airline,211.0,8.203056e+07,1.730845e+10,862.815186,182054.0
Mean_encoded_Airline,121.0,1.172841e+08,1.419138e+10,1384.000000,167464.0
Freq_encoded_Source,45.0,1.456858e+07,6.555862e+08,630.577759,28376.0
Freq_encoded_Route,280.0,1.219211e+07,3.413791e+09,914.432129,256041.0
Mean_encoded_Route,265.0,6.017502e+08,1.594638e+11,1321.022583,350071.0
Label_encoded_Airline,90.0,1.818257e+07,1.636431e+09,940.222229,84620.0
Label_encoded_Source,20.0,3.681148e+07,7.362296e+08,801.099976,16022.0
Label_encoded_Destination,18.0,7.722954e+06,1.390132e+08,1208.000000,21744.0


In [142]:
df_vif= sklearn_vif(exogs=X.columns, data=X).sort_values(by='VIF',ascending=False)

C:\Users\theve\AppData\Local\Temp/ipykernel_15328/2639988759.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_15328/2639988759.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_15328/2639988759.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_15328/2639988759.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_15328/2639988759.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_15328/2639988759.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
C:\Users\theve\AppData\Local\Temp/ipykernel_15328/2639988759.py:20: Ru

In [143]:
df_vif

,VIF,Tolerance
arrival_timeOfDay_night,inf,0.000000
departure_timeOfDay_morning,inf,0.000000
departure_timeOfDay_evening,inf,0.000000
departure_timeOfDay_early_morning,inf,0.000000
departure_timeOfDay_after_noon,inf,0.000000
arr_month,inf,0.000000
dep_month,inf,0.000000
departure_timeOfDay_night,inf,0.000000
arrival_timeOfDay_encoded,inf,0.000000
departure_timeOfDay_encoded,inf,0.000000


In [156]:
vif_cols = ['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 
#             'Freq_encoded_Source',
       'Freq_encoded_Destination', 'Freq_encoded_Route', 
#             'Mean_encoded_Source',
#        'Mean_encoded_Destination',
            'Mean_encoded_Route',
       'Label_encoded_Airline', 'Label_encoded_Source',
       'Label_encoded_Destination', 'Label_encoded_Route', 'dep_hr', 'arr_hr',
#        'departure_timeOfDay_encoded',
            'arrival_timeOfDay_encoded',
       'dep_day_of_week', 'arr_day_of_week', 'dep_weekday', 'arr_weekday',
       'dep_month', 'dep_day_of_month', 
#             'arr_month', 
#             'arr_day_of_month',
       'departure_timeOfDay_after_noon', 
#             'departure_timeOfDay_early_morning',
#        'departure_timeOfDay_evening',
            'departure_timeOfDay_late_night',
       'departure_timeOfDay_morning', 'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening', 
#             'arrival_timeOfDay_late_night',
#        'arrival_timeOfDay_morning', 
#             'arrival_timeOfDay_night'
           ]

X =X[vif_cols]

df_vif= sklearn_vif(exogs=X.columns, data=X).sort_values(by='VIF',ascending=False)
df_vif

,VIF,Tolerance
dep_weekday,9.876796,0.101247
arr_weekday,9.062172,0.110349
Total_Stops,7.229900,0.138315
dep_day_of_week,7.075572,0.141331
arr_day_of_week,6.621186,0.151030
Freq_encoded_Destination,6.219804,0.160777
Mean_encoded_Route,5.121795,0.195244
Label_encoded_Destination,3.546616,0.281959
Label_encoded_Route,3.466105,0.288508
Label_encoded_Source,2.556181,0.391209


In [157]:
model = train(X[vif_cols],y,xgboost)

importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.9476763358802425 0.8389300696701405


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


,weight,gain,total_gain,cover,total_cover
Total_Stops,76.0,1.357770e+07,1.031905e+09,327.052643,24856.0
Duration_min,1054.0,1.527122e+07,1.609586e+10,1146.655640,1208575.0
Freq_encoded_Airline,237.0,7.166138e+07,1.698375e+10,810.991577,192205.0
Mean_encoded_Airline,107.0,1.349155e+08,1.443595e+10,1951.476685,208808.0
Freq_encoded_Destination,57.0,1.051691e+07,5.994640e+08,865.298218,49322.0
Freq_encoded_Route,271.0,1.388248e+07,3.762153e+09,792.228760,214694.0
Mean_encoded_Route,240.0,6.604089e+08,1.584981e+11,1264.675049,303522.0
Label_encoded_Airline,73.0,1.527314e+07,1.114939e+09,1253.397217,91498.0
Label_encoded_Source,29.0,2.105186e+07,6.105039e+08,437.482758,12687.0
Label_encoded_Destination,14.0,6.534502e+06,9.148303e+07,922.071411,12909.0


## Cross Validation

In [163]:
from sklearn.model_selection import cross_validate

In [164]:
from xgboost import XGBRegressor
Xgboost_reg = XGBRegressor()

In [172]:
estimator = Xgboost_reg
X = X[vif_cols]
y = y
number_of_cross_validation = 5

def basic_cross_validation(estimator,X,y,number_of_cross_validation=5):
    scores = cross_validate(estimator, X, y, cv=number_of_cross_validation,
                             scoring=('r2', 'neg_mean_squared_error'),
                             return_train_score=True)

    print(scores['train_r2'])
    print(scores['train_r2'].mean())

    print(scores['test_r2'])
    print(scores['test_r2'].mean())

In [174]:
from catboost import CatBoostRegressor

In [175]:
basic_cross_validation(CatBoostRegressor(verbose=False),X[vif_cols],y,number_of_cross_validation=5)

[0.91783436 0.91659587 0.91639984 0.92123068 0.92181008]
0.9187741678298561
[0.86069031 0.84578275 0.87439976 0.86201367 0.84512159]
0.8576016155228118


#### Actions taken in feature Engineering

In [181]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)


def time_based_feature_Engineering(df):

    df['dep_hr'] = df['DepartureDateTime'].dt.hour
    df['arr_hr'] = df['ArrivalDateTime'].dt.hour

    df['dep_month'] = df['DepartureDateTime'].dt.month
    df['dep_day_of_month'] = df['DepartureDateTime'].dt.day

    df['arr_month'] = df['ArrivalDateTime'].dt.month
    df['arr_day_of_month'] = df['ArrivalDateTime'].dt.day

    df['dep_day_of_week'] = df['DepartureDateTime'].dt.day_of_week 
    df['arr_day_of_week'] = df['ArrivalDateTime'].dt.day_of_week 

    df['dep_weekday'] = np.where(df["dep_day_of_week"].isin([5,6]),0,1)
    df['arr_weekday'] = np.where(df["arr_day_of_week"].isin([5,6]),0,1)


    df['departure_timeOfDay'] = df['dep_hr'].apply(lambda x: time_of_day(x)[0])
    df['departure_timeOfDay_encoded'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
    df['arrival_timeOfDay'] = df['arr_hr'].apply(lambda x: time_of_day(x)[0])
    df['arrival_timeOfDay_encoded'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

    one_hot_cols = ['departure_timeOfDay','arrival_timeOfDay']
    df_oneHotEncoded = pd.get_dummies(df[one_hot_cols])
    new_df = pd.concat([df,df_oneHotEncoded],axis=1)
    
    drop_cols = [
                'DepartureDateTime',
                'Duration_timedelta',
                'ArrivalDateTime',
                'Freq_encoded_Source',
                'Mean_encoded_Source',
                'Mean_encoded_Destination',
                'departure_timeOfDay_encoded',
                'arr_month', 
                'arr_day_of_month',
                'departure_timeOfDay_early_morning',
                'departure_timeOfDay_evening',
                'arrival_timeOfDay_late_night',
                'arrival_timeOfDay_morning', 
                'arrival_timeOfDay_night']
    
    new_df.drop(columns=one_hot_cols,inplace=True)
    new_df.drop(columns=drop_cols,inplace=True)
    return new_df

In [182]:
final_df = time_based_feature_Engineering(df)

In [184]:
final_df.to_csv("../data/FinalData.csv",index=False)

In [183]:
final_df.columns

Index(['Total_Stops', 'Price', 'Duration_min', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 'Freq_encoded_Destination',
       'Freq_encoded_Route', 'Mean_encoded_Route', 'Label_encoded_Airline',
       'Label_encoded_Source', 'Label_encoded_Destination',
       'Label_encoded_Route', 'dep_hr', 'arr_hr', 'arrival_timeOfDay_encoded',
       'dep_day_of_week', 'arr_day_of_week', 'dep_weekday', 'arr_weekday',
       'dep_month', 'dep_day_of_month', 'departure_timeOfDay_after_noon',
       'departure_timeOfDay_late_night', 'departure_timeOfDay_morning',
       'departure_timeOfDay_night', 'arrival_timeOfDay_after_noon',
       'arrival_timeOfDay_early_morning', 'arrival_timeOfDay_evening'],
      dtype='object')

In [185]:
X = final_df.drop(columns='Price')
y = final_df['Price']
model = train(X,y,xgboost)

importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.9476763358802425 0.8389300696701405


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


,weight,gain,total_gain,cover,total_cover
Total_Stops,76.0,1.357770e+07,1.031905e+09,327.052643,24856.0
Duration_min,1054.0,1.527122e+07,1.609586e+10,1146.655640,1208575.0
Freq_encoded_Airline,237.0,7.166138e+07,1.698375e+10,810.991577,192205.0
Mean_encoded_Airline,107.0,1.349155e+08,1.443595e+10,1951.476685,208808.0
Freq_encoded_Destination,57.0,1.051691e+07,5.994640e+08,865.298218,49322.0
Freq_encoded_Route,271.0,1.388248e+07,3.762153e+09,792.228760,214694.0
Mean_encoded_Route,240.0,6.604089e+08,1.584981e+11,1264.675049,303522.0
Label_encoded_Airline,73.0,1.527314e+07,1.114939e+09,1253.397217,91498.0
Label_encoded_Source,29.0,2.105186e+07,6.105039e+08,437.482758,12687.0
Label_encoded_Destination,14.0,6.534502e+06,9.148303e+07,922.071411,12909.0


In [186]:
model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

## Paramter optimization

In [187]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [190]:
param_grid = {
    "learning_rate" : [0.1,0.2,0.3,0.5,],
    "max_depth" : [4,5,6,7],
    "n_estimators" : [100,200,300,400,500]
}

In [191]:
reg = RandomizedSearchCV(XGBRegressor(),param_grid,random_state=0)

In [192]:
model = reg.fit(X,y)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

In [193]:
model.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.2, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [195]:
model

RandomizedSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          enable_categorical=False, gamma=None,
                                          gpu_id=None, importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n_jobs=None,
                                          num_parallel_tree=None,
                                          predictor=None, random_state=None,
                                          reg_alpha=None, reg_lambda=None,
                                          scale_pos_weight=None, subsample=None,
                                          tree_method=None,
                                          validate_parameters=None,
                                          verbosity=None),
                   param_distributions={'learning_rate': [0.1, 0.2, 0.3, 0.5],
                                        'max_depth': [4, 5, 6, 7],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   random_state=0)

In [194]:
basic_cross_validation(model.best_estimator_,X,y,number_of_cross_validation=5)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

[0.92641245 0.92785294 0.92490874 0.9312887  0.92737507]
0.9275675803339741
[0.85132166 0.83736161 0.87013503 0.85846186 0.84702403]
0.8528608377944492


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [197]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
xgboost_regressor = model.best_estimator_
xgboost_regressor.fit(X_train, y_train)
print(metrics.r2_score(y_train,xgboost_regressor.predict(X_train)),metrics.r2_score(y_test,xgboost_regressor.predict(X_test)))

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.938191536955651 0.847452620050444


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [198]:
param_grid = {
    "learning_rate" : [0.2,0.21,0.29],
    "n_estimators" : [95,100,105]
}

In [200]:
reg = GridSearchCV(XGBRegressor(),param_grid)

In [201]:
model = reg.fit(X,y)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

In [202]:
model.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.21, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [203]:
model

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                                    subsample=None, tree_method=None,
                                    validate_parameters=None, verbosity=None),
             param_grid={'learning_rate': [0.2, 0.21, 0.29],
                         'n_estimators': [95, 100, 105]})

In [204]:
basic_cross_validation(model.best_estimator_,X,y,number_of_cross_validation=5)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

[0.92834462 0.93043533 0.92751398 0.93373321 0.92853034]
0.9297114941441
[0.85332205 0.84010909 0.87319836 0.85668944 0.8529241 ]
0.8552486070259919


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [205]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
xgboost_regressor = model.best_estimator_
xgboost_regressor.fit(X_train, y_train)
print(metrics.r2_score(y_train,xgboost_regressor.predict(X_train)),metrics.r2_score(y_test,xgboost_regressor.predict(X_test)))

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.9370925362707928 0.8462210479716865


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [ ]:
# XGBRegressor( learning_rate=0.21, max_depth=6,  n_estimators=100)

In [206]:
import pickle

In [213]:
with open('../models/xgboost.pickle','wb') as file:
        pickle.dump(xgboost_regressor,file)